# Implementing players

In [1]:
from abc import ABC, abstractmethod
import numpy as np

In [ ]:
class Player(ABC):
    
    GK = 0
    DEF = 1
    MID = 2
    ATT = 3
    
    SHOOT = 'shoots'
    PASS = 'passes'
    
    pass_mod = {
        GK: {DEF: 0, MID: 0.2, ATT: 0.3},
        DEF: {GK: 0, DEF: 0, MID: 0.1, ATT: 0.3},
        MID: {GK: 0.2, DEF: 0.1, MID: 0, ATT: 0.1},
        ATT: {GK: 0.5, DEF: 0.4, MID: 0.2, ATT: 0},
    }
    
    @abstractmethod
    def initialize(self, data):
        self.id = data['ID']
        self.name = data['name']
        self.club = data['club']
        self.league = data['league']
        self.overall = data['overall']
        self.psg = data['psg']
        self.attack = data['atk']
        self.defense = data['dfs']
        self.preferences = {Player.GK: 0.25, Player.DEF: 0.25, 
                            Player.MID: 0.25, Player.ATT: 0.25}
        self._shoot = 0.5
        
    def shoot(self):
        pass
    
    def play(self, options):
        if np.random.uniform() < self._shoot:
            return self.shoot()
        else:
            return self.passing(options)
    
    def passing(self, options):
        p = np.array([self.preferences[o.role] if o.id != self.id else 0 for o in options])
        receiver = np.random.choice(options, p=p / p.sum())
        outcome = np.random.uniform() <= self.psg - Player.pass_mod[self.role][receiver.role]
        return Player.PASS, outcome, receiver

    def __repr__(self):
        return "{}, {}, A: {}, D: {}, P: {}".format(self.name, 
                                                    self.role, self.attack, 
                                                    self.defense, self.psg)
        
class GK(Player):
    
    def __init__(self, data):
        self.initialize(data)
        self.defense = data['gkv']
        self.preferences = {Player.GK: 0.0, Player.DEF: 0.5, Player.MID: 0.25, Player.ATT: 0.25}
        self._shoot = 0.0
        self.role = Player.GK

    def initialize(self, data):
        super().initialize(data)
    
    def shoot(self):
        return Player.SHOOT, False, None
    
    def passing(self, options):
        return super().passing(options)
    
    def play(self, options):
        return super().play(options)
    
    def saves(self):
        return np.random.uniform() <= self.defense
        
        
class Defender(Player):
    
    def __init__(self, data):
        self.initialize(data)
        self.preferences = {Player.GK: 0.25, 
                            Player.DEF: 0.4, Player.MID: 0.25, 
                            Player.ATT: 0.1}
        self._shoot = 0.1
        self.role = Player.DEF

    def initialize(self, data):
        super().initialize(data)
    
    def shoot(self):
        return Player.SHOOT, np.random.uniform() <= self.attack / 1.5, None
        
    def defend(self):
        return np.random.uniform() <= self.defense
        
    def passing(self, options):
        return super().passing(options)
        
    def play(self, options):
        return super().play(options)
        

class Midfielder(Player):
    
    def __init__(self, data):
        self.initialize(data)
        self.preferences = {Player.GK: 0.05, Player.DEF: 0.25, Player.MID: 0.4, Player.ATT: 0.3}
        self._shoot = 0.3
        self.role = Player.MID

    def initialize(self, data):
        super().initialize(data)
        
    def shoot(self):
        return Player.SHOOT, np.random.uniform() <= self.attack / 1.2, None
    
    def defend(self):
        return np.random.uniform() <= self.defense / 1.2
        
    def passing(self, options):
        return super().passing(options)
    
    def play(self, options):
        return super().play(options)
        
        
class Attacker(Player):
    
    def __init__(self, data):
        self.initialize(data)
        self.preferences = {Player.GK: 0.0, Player.DEF: 0.05, Player.MID: 0.35, Player.ATT: 0.6}
        self._shoot = 0.5
        self.role = Player.ATT

    def initialize(self, data):
        super().initialize(data)

    def shoot(self):
        return Player.SHOOT, np.random.uniform() <= self.attack, None

    def defend(self):
        return np.random.uniform() <= self.defense / 1.5
        
    def passing(self, options):
        return super().passing(options)
    
    def play(self, options):
        return super().play(options)
        
        
class PlayerCreator(ABC):
    
    role_map = {
        'gk': GK,
        'ram': Midfielder,
        'rb': Defender,
        'rcb': Defender,
        'rcm': Midfielder,
        'rdm': Midfielder,
        'rf': Attacker,
        'rm': Midfielder,
        'rs': Attacker,
        'rw': Attacker,
        'rwb': Defender
    }

    @staticmethod
    def create(data):
        try:
            return PlayerCreator.role_map[data['role']](data)
        except KeyError:
            print ("Role {} not available".format(data['role']))
            return None